<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/main/notebooks/Analyse%20CH%20Compare%20Full%20Time%20Work%20Percentage%20For%20Wealthiest%20And%20Poorest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

#Analyse CH Compare Full Time Work Percentage For Wealthiest And Poorest

*Instructions:*

1. Execute the first code cell.
2. There will be a link to follow in order to authorize the google account for drive. Go to that link.
3. A code to authorize the google account will be generated. Copy the code generated.
4. Go back to the cell where the process of mounting the drive is running. Paste the generated code from step 3 to the text box in the cell and press enter.



In [ ]:
# add time information at the end of every cell
!pip install ipython-autotime
%load_ext autotime

time: 3 ms (started: 2021-04-22 02:05:00 +00:00)


In [ ]:
# Install required dependencies
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 67kB/s 
     |████████████████████████████████| 204kB 42.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=42aac568554992d2bec60def6f48952078955d01bcc5cce3c242aa2efaf8d84a
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [ ]:
# Import modules
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 100 ms (started: 2021-04-22 02:05:52 +00:00)


In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 244 ms (started: 2021-04-22 02:05:52 +00:00)


In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 4.67 s (started: 2021-04-22 02:05:52 +00:00)


In [ ]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 25.2 s (started: 2021-04-22 02:05:57 +00:00)


In [ ]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'ch'

time: 2.59 ms (started: 2021-04-22 02:06:22 +00:00)


In [ ]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [ ]:
#Create view for sparksql
df_soc.createOrReplaceTempView('clean_ch')
df_soc = spark.sql('SELECT cbg, date_range_start as date, device_count, full_time_work_behavior_devices FROM clean_ch WHERE device_count > 5')
df_soc.show()

+------------+--------------------+------------+-------------------------------+
|         cbg|                date|device_count|full_time_work_behavior_devices|
+------------+--------------------+------------+-------------------------------+
|170312304002|2020-02-07T00:00:...|          76|                              7|
|170313005002|2020-02-07T00:00:...|          72|                              9|
|170314208001|2020-02-07T00:00:...|         127|                             15|
|170317608034|2020-02-07T00:00:...|         114|                             10|
|170318033003|2020-02-07T00:00:...|          91|                             13|
|170318146005|2020-02-07T00:00:...|          51|                              4|
|170318182002|2020-02-07T00:00:...|          51|                              1|
|170318240053|2020-02-07T00:00:...|         123|                             13|
|170318348001|2020-02-07T00:00:...|          32|                              6|
|170438411042|2020-02-07T00:

In [ ]:
# Taking substring of date for "yyyy-mm-dd"
df_soc = df_soc.withColumn("date",
    df_soc['date'].substr(0, 10))

time: 55.6 ms (started: 2021-04-22 02:06:46 +00:00)


In [ ]:
df_soc.show()

+------------+----------+------------+-------------------------------+
|         cbg|      date|device_count|full_time_work_behavior_devices|
+------------+----------+------------+-------------------------------+
|170312304002|2020-02-07|          76|                              7|
|170313005002|2020-02-07|          72|                              9|
|170314208001|2020-02-07|         127|                             15|
|170317608034|2020-02-07|         114|                             10|
|170318033003|2020-02-07|          91|                             13|
|170318146005|2020-02-07|          51|                              4|
|170318182002|2020-02-07|          51|                              1|
|170318240053|2020-02-07|         123|                             13|
|170318348001|2020-02-07|          32|                              6|
|170438411042|2020-02-07|          57|                              5|
|170438427041|2020-02-07|          84|                              6|
|18089

In [ ]:
# Add full time work percentage column

df_soc = df_soc.withColumn('full_time_work_percentage', (df_soc['full_time_work_behavior_devices']/df_soc['device_count']) * 100)

time: 66.2 ms (started: 2021-04-22 02:06:46 +00:00)


In [ ]:
#Create temp view
df_soc.createOrReplaceTempView('mobility')

time: 19.1 ms (started: 2021-04-22 02:06:46 +00:00)




---


**Demographic data**





---



In [ ]:
# Read poverty data
# Read data into RDD
df_demographic = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'ch.csv'))
df_demographic.createOrReplaceTempView('demographic')
df_demographic.show()

+------------+---------+------------------+-------------------+
|         cbg|pop_total|poverty_percentage|     perc_whiteonly|
+------------+---------+------------------+-------------------+
|170312909001|      935|  55.8288770053476|                0.0|
|170312909002|     1129|45.261293179805136|                0.0|
|170312909003|      765|  39.7078353253652|                0.0|
|170312909004|     1075|30.325581395348838|                0.0|
|170318387002|     1016|31.003937007874015| 13.484251968503939|
|170313102001|      835|16.766467065868262|  78.20359281437126|
|170313102002|      686| 16.61807580174927| 50.583090379008745|
|170314005002|      807|40.644361833952914|                0.0|
|170314804006|      710| 1.971830985915493|  1.267605633802817|
|170314804007|      650|               0.0|                0.0|
|170314805001|     1141|  9.55302366345311|  4.206836108676599|
|170312608002|     1100| 55.45454545454545| 2.8181818181818183|
|170312609001|      836| 44.856459330143

In [ ]:
# Performing inner join on mobility and demographic table using cbg
df_mob_demo = spark.sql('SELECT mobility.*, demographic.poverty_percentage from mobility INNER JOIN demographic ON mobility.cbg = demographic.cbg')
df_mob_demo.createOrReplaceTempView('demographic_mobility')
df_mob_demo.show()


+------------+----------+------------+-------------------------------+-------------------------+------------------+
|         cbg|      date|device_count|full_time_work_behavior_devices|full_time_work_percentage|poverty_percentage|
+------------+----------+------------+-------------------------------+-------------------------+------------------+
|170312304002|2020-02-07|          76|                              7|        9.210526315789473|19.847972972972975|
|170313005002|2020-02-07|          72|                              9|                     12.5|22.696011004126547|
|170314208001|2020-02-07|         127|                             15|       11.811023622047244| 34.01677539608574|
|170317608034|2020-02-07|         114|                             10|        8.771929824561402|13.651647612642906|
|170318033003|2020-02-07|          91|                             13|       14.285714285714285| 4.035608308605341|
|170318146005|2020-02-07|          51|                              4|  

In [ ]:
# DIVIDING THE PEOPLE TOP 20 PERCENT AND BOTTOM 20 PERCENT

df_mob_demo_high_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage < 20')
grouped_df_mob_demo_high_class = df_mob_demo_high_class.groupBy("date").mean("full_time_work_percentage").withColumnRenamed('avg(full_time_work_percentage)','full_time_work_percentage')
grouped_df_mob_demo_high_class.createOrReplaceTempView('mob_demo_high_class')


df_mob_demo_low_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage > 80')
grouped_df_mob_demo_low_class = df_mob_demo_low_class.groupBy("date").mean("full_time_work_percentage").withColumnRenamed('avg(full_time_work_percentage)','full_time_work_percentage')
grouped_df_mob_demo_low_class.createOrReplaceTempView('mob_demo_low_class')

grouped_df_mob_demo_combined = spark.sql("""

SELECT h.date,h.full_time_work_percentage as top20_perc,
l.full_time_work_percentage as bottom20_perc  
FROM mob_demo_high_class h INNER JOIN mob_demo_low_class l ON h.date = l.date

""")

time: 305 ms (started: 2021-04-22 02:06:48 +00:00)


**Load 2019 stats**

In [ ]:
# Getting mean of 2019 value for part time device count
filename = '{}_social_summary_2019.csv'.format(city)
stat_2019 = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/filename))
stat_2019.show()
mean_2019 = stat_2019.collect()[1][7]


+-----+------------------+------------------+------------------+-----------------+--------------------------+-------------------------+-------------------------+--------------------+--------------------+--------------------+
|  _c0|             total|   completely_home|    part_time_work|   full_time_work|percentage_completely_home|percentage_part_time_work|percentage_full_time_work|norm_completely_home| norm_part_time_work| norm_full_time_work|
+-----+------------------+------------------+------------------+-----------------+--------------------------+-------------------------+-------------------------+--------------------+--------------------+--------------------+
|count|         2430227.0|         2430227.0|         2430227.0|        2430227.0|                 2430227.0|                2430227.0|                2430227.0|           2430227.0|           2430227.0|           2430227.0|
| mean| 95.04381072220825|29.833984232748627|  8.54960421392734|5.511380212630343|        30.9662652

In [ ]:
# calculate deviation form mean
grouped_df_mob_demo_combined= grouped_df_mob_demo_combined.withColumn("top20_deviation",
    grouped_df_mob_demo_combined['top20_perc'] - mean_2019)


time: 35.4 ms (started: 2021-04-22 02:06:49 +00:00)


In [ ]:
# Taking bottom and top 20 percent to get the wealthiest and poorest population data
grouped_df_mob_demo_combined = grouped_df_mob_demo_combined.withColumn("bottom20_deviation",
    grouped_df_mob_demo_combined['bottom20_perc'] - mean_2019)

time: 36.7 ms (started: 2021-04-22 02:06:49 +00:00)



**Visualizations** 



In [ ]:
# create altair library theme 

def my_theme():
  return {
    'config': {
      'view': {'continuousHeight': 300, 'continuousWidth': 400},  # from the default theme
      'range': {'category': {'scheme': ['#FF4500','#a0aab4','#4E79A7']}}
    }
  }
alt.themes.register('my_theme', my_theme)
alt.themes.enable('my_theme')
# 

ThemeRegistry.enable('my_theme')

time: 18.1 ms (started: 2021-04-22 02:06:49 +00:00)


In [ ]:
# Define characteristics for visualisation creation
def create_chart(df) :
  chart = alt.Chart(df).transform_fold(
    fold=['Bottom 20 % Deviation', 'Top 20 % Deviation'],
    as_ = ['Poverty %','full_time_work']
    
    ).mark_line().encode(
      x= alt.X('date:T', title = 'Date'),
      y= alt.Y('full_time_work:Q', title = 'Full Time %'),
      color='Poverty %:N'
    ).properties(width = 1000).interactive()

  rule = alt.Chart(pd.DataFrame({'Mean 2019':[0]})).mark_rule(color='#757575',strokeDash=[5,3], size=2).encode(
    y='Mean 2019',
    size=alt.value(2),
    )
  
  text = alt.Chart({'values':[{ 'y': 1}]}).mark_text(
    text='Mean 2019', angle=0
    ).encode(
      y=alt.Y('y:Q'),opacity=alt.value(0.4)
    )

  return  chart + rule + text


time: 13.9 ms (started: 2021-04-22 02:06:49 +00:00)


In [ ]:
# Function to filter date from dataset
def apply_date_filter(df,start_date,end_date) :
  mask = (df['date'] > start_date) & (df['date'] <= end_date)
  dfX = df.loc[mask]
  return dfX

time: 6.17 ms (started: 2021-04-22 02:06:49 +00:00)


In [ ]:
# create pandas dataframe for visualizations
df = grouped_df_mob_demo_combined.toPandas()
df

,date,top20_perc,bottom20_perc,top20_deviation,bottom20_deviation
0,2020-02-26,9.570720,4.370398,3.572131,-1.628191
1,2020-04-13,3.230145,4.072505,-2.768444,-1.926084
2,2020-06-24,3.839379,3.734758,-2.159210,-2.263832
3,2020-06-08,3.947207,2.657462,-2.051383,-3.341128
4,2020-09-12,2.379733,3.036618,-3.618856,-2.961972
...,...,...,...,...,...
361,2020-04-05,2.053843,2.609683,-3.944746,-3.388906
362,2020-05-01,4.069700,6.172795,-1.928890,0.174206
363,2020-10-25,2.340273,3.510834,-3.658316,-2.487756
364,2020-12-28,5.297158,6.121226,-0.701431,0.122637


time: 4min 3s (started: 2021-04-22 02:06:49 +00:00)


In [ ]:
df = df.rename(columns={'top20_perc': 'Top 20 %', 'bottom20_perc': 'Bottom 20 %','top20_deviation': 'Top 20 % Deviation', 'bottom20_deviation': 'Bottom 20 % Deviation'})

time: 2.97 ms (started: 2021-04-22 02:10:53 +00:00)


In [ ]:
# analysis for the entire year on monthly basis
df_str = df.copy(deep=True) #make a deep copy

df_str['date'] = pd.to_datetime(df_str['date'],utc= True) # extract the months

df_str = df_str.groupby([df_str['date']]).mean().rolling(10).mean()[0:] #find the mean for the days and roll up for every 10 days

chart = alt.Chart(df_str.reset_index()).transform_fold(
    fold=['Bottom 20 %', 'Top 20 %'],
    as_ = ['Poverty %','full_time_percentage']
    
    ).mark_line().encode(
      x='date:T',
      y= alt.Y('full_time_percentage:Q', title = 'Full Time %'), color='Poverty %:N'
    ).properties(width = 1000)
chart

alt.Chart(...)

time: 137 ms (started: 2021-04-22 02:10:53 +00:00)


In [ ]:
#Conversion of datetime format
df['date'] = pd.to_datetime(df['date'],utc= True)
 

time: 7.15 ms (started: 2021-04-22 02:10:53 +00:00)


#Filtering based on important dates

**March 17, 2020:**
Governor Pritzker announced statewide school closures beginning March 17 until March 30.












In [ ]:
start_date = '2020-3-12'
end_date = '2020-4-3'
df1 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df1)
chart

alt.LayerChart(...)

time: 110 ms (started: 2021-04-22 02:10:53 +00:00)




**April 20, 2020:**
Illinois state topped 3,000 new cases in a day for the first time. 









In [ ]:
start_date = '2020-4-14'
end_date = '2020-5-1'
df2 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df2)
chart
#df2

alt.LayerChart(...)

time: 79.9 ms (started: 2021-04-22 02:10:53 +00:00)



**June 3, 2020:** 
Chicago began gradually allowing businesses to open doors for outside dining and other services, though some shops remained closed following lootings in the aftermath of George Floyd's death, during phase three of its reopening.




In [ ]:
start_date = '2020-6-1'
end_date = '2020-6-16'
df3 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df3)
chart

alt.LayerChart(...)

time: 53.3 ms (started: 2021-04-22 02:10:53 +00:00)


**September 8, 2020:** Chicago Public Schools began its fall quarter with remote learning, with families across the city reporting a range of successes and frustrations.


In [ ]:
start_date = '2020-09-1'
end_date = '2020-09-14'
df4 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df4)
chart

alt.LayerChart(...)

time: 48.9 ms (started: 2021-04-22 02:10:53 +00:00)
